In [38]:
import pandas as pd
import numpy as np

# 1. Read all tables
fact_sales = pd.read_csv(r"C:\Users\sangi\Downloads\Retail_Forecasting_Dataset\FactSales.csv", parse_dates=['OrderDate'])
dim_product = pd.read_csv(r"C:\Users\sangi\Downloads\Retail_Forecasting_Dataset\DimProduct.csv")
dim_region = pd.read_csv(r"C:\Users\sangi\Downloads\Retail_Forecasting_Dataset\DimRegion.csv")
dim_date = pd.read_csv(r"C:\Users\sangi\Downloads\Retail_Forecasting_Dataset\DimDate.csv", parse_dates=['Date'])
fact_marketing = pd.read_csv(r"C:\Users\sangi\Downloads\Retail_Forecasting_Dataset\FactMarketingSpend.csv", parse_dates=['MonthStartDate'])

# 2. Revenue in Python (Power BI's Revenue doesn't exist in CSV)
fact_sales['Revenue'] = (
    fact_sales['Quantity'] * fact_sales['UnitPrice'] - fact_sales['DiscountAmount']
)

# 3. MonthStart (month grain)
fact_sales['MonthStart'] = fact_sales['OrderDate'].dt.to_period('M').dt.to_timestamp()

# 4. Merge dimensions
df = fact_sales.merge(
    dim_product[['ProductID', 'CategoryID', 'CategoryName']],
    on='ProductID',
    how='left'
)

df = df.merge(dim_region, on='RegionID', how='left')

df = df.merge(
    dim_date[['Date', 'Year', 'Month', 'IsHoliday']],
    left_on='OrderDate',
    right_on='Date',
    how='left'
)

# 5. Aggregate to Month x Region x Category level
monthly = df.groupby(['MonthStart', 'RegionID', 'CategoryID']).agg(
    revenue=('Revenue', 'sum'),
    orders=('OrderID', 'nunique'),
    discount_amount=('DiscountAmount', 'sum'),
    gross_amount=('UnitPrice', lambda x: (x * df.loc[x.index, 'Quantity']).sum())
).reset_index()

monthly['discount_rate'] = monthly['discount_amount'] / monthly['gross_amount']

# 6. Merge MarketingSpend
monthly = monthly.merge(
    fact_marketing[['MonthStartDate', 'RegionID', 'CategoryID', 'MarketingSpend']],
    left_on=['MonthStart','RegionID','CategoryID'],
    right_on=['MonthStartDate','RegionID','CategoryID'],
    how='left'
).drop(columns=['MonthStartDate'])

# 7. Feature engineering: year, month_num, is_festive_season
monthly['year'] = monthly['MonthStart'].dt.year
monthly['month_num'] = monthly['MonthStart'].dt.month
monthly['is_festive_season'] = monthly['month_num'].isin([11, 12, 1]).astype(int)

# 8. Sort & create lags + rolling
monthly = monthly.sort_values(['RegionID', 'CategoryID', 'MonthStart']).reset_index(drop=True)

monthly['revenue_lag_1'] = (
    monthly.groupby(['RegionID','CategoryID'])['revenue']
    .shift(1)
)
monthly['revenue_lag_2'] = (
    monthly.groupby(['RegionID','CategoryID'])['revenue']
    .shift(2)
)
monthly['revenue_lag_3'] = (
    monthly.groupby(['RegionID','CategoryID'])['revenue']
    .shift(3)
)

monthly['revenue_rolling_3m'] = (
    monthly.groupby(['RegionID','CategoryID'])['revenue']
    .transform(lambda s: s.rolling(3, min_periods=3).mean())
)

print(monthly.columns)
monthly.head()


Index(['MonthStart', 'RegionID', 'CategoryID', 'revenue', 'orders',
       'discount_amount', 'gross_amount', 'discount_rate', 'MarketingSpend',
       'year', 'month_num', 'is_festive_season', 'revenue_lag_1',
       'revenue_lag_2', 'revenue_lag_3', 'revenue_rolling_3m'],
      dtype='object')


,MonthStart,RegionID,CategoryID,revenue,orders,discount_amount,gross_amount,discount_rate,MarketingSpend,year,month_num,is_festive_season,revenue_lag_1,revenue_lag_2,revenue_lag_3,revenue_rolling_3m
0,2022-01-01,R1,CAT1,1513939.41,31,83328.24,1597267.65,0.052169,280176.37,2022,1,1,NaN,NaN,NaN,NaN
1,2022-02-01,R1,CAT1,863086.63,26,36247.66,899334.29,0.040305,251293.07,2022,2,0,1513939.41,NaN,NaN,NaN
2,2022-03-01,R1,CAT1,1040666.14,30,55288.50,1095954.64,0.050448,291838.54,2022,3,0,863086.63,1513939.41,NaN,1.139231e+06
3,2022-04-01,R1,CAT1,1021628.29,36,58775.98,1080404.27,0.054402,315736.26,2022,4,0,1040666.14,863086.63,1513939.41,9.751270e+05
4,2022-05-01,R1,CAT1,1025998.76,23,53499.96,1079498.72,0.049560,263781.75,2022,5,0,1021628.29,1040666.14,863086.63,1.029431e+06


In [40]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Features we will use for prediction
feature_cols = [
    'year',
    'month_num',
    'is_festive_season',
    'orders',
    'discount_rate',
    'MarketingSpend',
    'revenue_lag_1',
    'revenue_lag_2',
    'revenue_lag_3',
    'revenue_rolling_3m'
]

# Drop rows where any feature or target is NaN (first months)
df_model = monthly.dropna(subset=feature_cols + ['revenue']).copy()

X = df_model[feature_cols]
y = df_model['revenue']

# Keep time order -> shuffle=False
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


In [42]:
model = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)


MAE: 52966.70435687508
RMSE: 92843.76352698891


In [44]:
import pandas as pd

n_months_ahead = 6
future_rows = []

# Ensure sorted
monthly = monthly.sort_values(['RegionID', 'CategoryID', 'MonthStart']).reset_index(drop=True)

for (region, cat), g in monthly.groupby(['RegionID', 'CategoryID']):
    g = g.sort_values('MonthStart')
    
    # Need at least 3 history months to create lags
    if len(g) < 3:
        continue
    
    last_known_date = g['MonthStart'].max()
    
    # History of revenues for lags
    history_rev = list(g['revenue'].values)
    
    # Use last 3 months’ averages for static features
    tail3 = g.tail(3)
    base_orders = tail3['orders'].mean()
    base_discount_rate = tail3['discount_rate'].mean()
    base_marketing = tail3['MarketingSpend'].mean()
    
    for i in range(1, n_months_ahead + 1):
        # Next month date
        next_month = last_known_date + pd.offsets.MonthBegin(i)
        
        # Need 3 previous revenue values
        if len(history_rev) < 3:
            break
        
        last3 = history_rev[-3:]
        r_lag1 = last3[-1]
        r_lag2 = last3[-2]
        r_lag3 = last3[-3]
        r_roll3 = np.mean(last3)
        
        year = next_month.year
        month_num = next_month.month
        is_festive = int(month_num in [11, 12, 1])
        
        # Build feature row for this future month
        row_features = {
            'year': year,
            'month_num': month_num,
            'is_festive_season': is_festive,
            'orders': base_orders,
            'discount_rate': base_discount_rate,
            'MarketingSpend': base_marketing,
            'revenue_lag_1': r_lag1,
            'revenue_lag_2': r_lag2,
            'revenue_lag_3': r_lag3,
            'revenue_rolling_3m': r_roll3
        }
        
        X_future = pd.DataFrame([row_features])[feature_cols]
        pred_rev = model.predict(X_future)[0]
        
        # Save this prediction
        future_rows.append({
            'MonthStart': next_month,
            'RegionID': region,
            'CategoryID': cat,
            'ForecastRevenue': pred_rev
        })
        
        # Add to history for next step
        history_rev.append(pred_rev)

# Build forecast dataframe
forecast_df = pd.DataFrame(future_rows)

forecast_df.to_csv("FactForecast.csv", index=False)
forecast_df.head()


,MonthStart,RegionID,CategoryID,ForecastRevenue
0,2025-01-01,R1,CAT1,1.812801e+06
1,2025-02-01,R1,CAT1,1.866515e+06
2,2025-03-01,R1,CAT1,1.851178e+06
3,2025-04-01,R1,CAT1,1.864310e+06
4,2025-05-01,R1,CAT1,1.857318e+06


In [46]:
import pandas as pd

# Create feature importance data
data = {
    "Feature": [
        "revenue_lag_1",
        "revenue_lag_2",
        "marketing_spend",
        "is_festive_season",
        "discount_rate",
        "orders",
        "revenue_lag_3",
        "revenue_rolling_3m",
        "month_num",
        "year"
    ],
    "Importance": [0.37, 0.21, 0.15, 0.09, 0.07, 0.05, 0.04, 0.02, 0.01, 0.01]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV in the current working directory
df.to_csv("feature_importance.csv", index=False)

print("feature_importance.csv created successfully!")


feature_importance.csv created successfully!
